In [ ]:
# Get the data into google drive
!wget http://128.138.93.164/meta_Clothing_Shoes_and_Jewelry.json.gz

--2023-03-19 19:19:15--  http://128.138.93.164/meta_Clothing_Shoes_and_Jewelry.json.gz
Connecting to 128.138.93.164:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 279748879 (267M) [application/octet-stream]
Saving to: ‘meta_Clothing_Shoes_and_Jewelry.json.gz’

meta_Clothing_Shoes 100%[===================>] 266.79M  60.1MB/s    in 4.7s    

2023-03-19 19:19:20 (56.4 MB/s) - ‘meta_Clothing_Shoes_and_Jewelry.json.gz’ saved [279748879/279748879]



In [ ]:
!wget http://128.138.93.164/reviews_Clothing_Shoes_and_Jewelry.json.gz

--2023-03-19 19:19:37--  http://128.138.93.164/reviews_Clothing_Shoes_and_Jewelry.json.gz
Connecting to 128.138.93.164:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 888065454 (847M) [application/octet-stream]
Saving to: ‘reviews_Clothing_Shoes_and_Jewelry.json.gz’

reviews_Clothing_Sh 100%[===================>] 846.92M  59.6MB/s    in 15s     

2023-03-19 19:19:51 (57.9 MB/s) - ‘reviews_Clothing_Shoes_and_Jewelry.json.gz’ saved [888065454/888065454]



In [ ]:
!python -m spacy download en_core_web_md

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-03-19 20:20:41.893969: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-19 20:20:41.894118: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-19 20:20:41.894145: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentione

In [ ]:
import gzip
import json
from itertools import chain
import pandas as pd
import spacy

## Getting all ASIN that contain Nike as a category

In [ ]:
# Get ASIN for products that have "Nike" as a category item
asins = []
with gzip.open("/content/meta_Clothing_Shoes_and_Jewelry.json.gz") as products:
    for product in products:
        data = eval(product)
        if "nike" in {x.lower() for x in chain(*data.get('categories', []))}:
            asins.append(data["asin"])

In [ ]:
len(asins)

8327

## Getting reviews for all gathered ASIN

In [ ]:
# Get reviews matching the ASINs we collected
review_collection = []
with gzip.open("/content/reviews_Clothing_Shoes_and_Jewelry.json.gz") as reviews:
    for review in reviews:
        data = json.loads(review)
        if data['asin'] in asins:
            review_collection.append(data)

In [ ]:
reviews_df = pd.DataFrame.from_dict(review_collection)

## Some Data Cleaning

In [ ]:
#Load a medium trained model
nlp = spacy.load("en_core_web_md")

In [ ]:
def cleaner(note):
    """Remove stop words, punctuations, convert words into lemma_"""
    doc = nlp(note)
    note = (" ".join([token.lemma_ for token in doc \
                      if not token.is_stop \
                          and not token.is_punct \
                          and not token.like_num\
                          and not token.like_url\
                          and not token.like_email]))
    note = note.replace('@', '')
    note = note.lower()
    return note

In [ ]:
reviews_df['cleaned_review'] = reviews_df['reviewText'].apply(cleaner)

### View a sample of the cleaned data

In [ ]:
with pd.option_context("display.max_colwidth", None):
  display(reviews_df[['reviewText', 'cleaned_review']].sample(10))

,reviewText,cleaned_review
9059,"Very comfortable, very stylish, great price. More comfortable than the nike free runs in my opinion. Looks good with jeans or khakis.",comfortable stylish great price comfortable nike free run opinion look good jean khaki
21391,"I was an air max LTD guy for years, because of the comfort level and the fact the look great with anything.I bought these because of the many reviews out there, stating how great these are, and the fact that nike only makes ugly and I mean ugly LTDs no a days. These exceeded my expectations by a long shot, the comfort level is better than any Ltd or runner I have owned in my life. I will order another pair!They are pricey but they are seriously worth every penny...Buy them you will Thank me.",air max ltd guy year comfort level fact look great buy review state great fact nike make ugly mean ugly ltd day exceed expectation long shot comfort level well ltd runner own life order pair!they pricey seriously worth penny buy thank
15166,"Well, to start off i didn't buy these shoes from amazon i got them at a different store. The shoes are comfy and good looking they look great on both jeans and shorts, they are well made and diffenetly worth the price even though the materials used now days are worse than back then xD.",start buy shoe amazon get different store shoe comfy good look look great jean short diffenetly worth price material day bad xd.
8901,"My daughter is a star athlete & she loves this bag! This bag holds just about anything she can think of. It's a quality product sold at a fair price & was delivered in a short amount of time. Thumbs up all around on this bag, homies!",daughter star athlete love bag bag hold think quality product sell fair price deliver short time thumb bag homie
17446,These have no break-in time. Comfortable as soon as you put them on.A very good price for these shoes.,break time comfortable soon good price shoe
5750,I bought these shoes for my niece. She loves them! They were priced well and the quality of the shoe is good. Great buy!,buy shoe niece love price quality shoe good great buy
7392,This bag is made of very sturdy materials. Perfect size for one change of clothes or shoes.Priced lower than any we've seen in retail stores.,bag sturdy material perfect size change clothe shoe price low see retail store
6287,These were purchased as a pair of shoes to use only at the gym. They are comfortable and support my feet as all of my previous Nike shoes have.,purchase pair shoe use gym comfortable support foot previous nike shoe
15847,Shoes fit and feel great. They run small so I ordered a half size bigger which cannot be found in the store (in a size 12.5).,shoe fit feel great run small order half size big find store size
19508,Great shoe for running or if you have a full day of walking planned. Well built shoe and will last a long time.,great shoe running day walking plan build shoe long time


In [ ]:
# Cleaner format for the review time 
reviews_df['reviewTime'] = pd.to_datetime(reviews_df['reviewTime'])

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

## Model using LDA

In [ ]:
counter = CountVectorizer()
bag = counter.fit_transform(reviews_df['cleaned_review'].values)

In [ ]:
# Bag contains a sparse matix of all document, word combinations as a 1-gram model (broken off by single words)
bag.toarray().shape

(21570, 16410)

In [ ]:
number_of_topics = 20

lda = LatentDirichletAllocation(
    n_components=number_of_topics,
    doc_topic_prior=0.02, # alpha
    topic_word_prior=0.01, # eta
    max_iter=20,
    learning_offset=20,
    n_jobs=-1
)

In [ ]:
lda.fit(bag)

LatentDirichletAllocation(doc_topic_prior=0.02, learning_offset=20, max_iter=20,
                          n_components=20, n_jobs=-1, topic_word_prior=0.01)

In [ ]:
predicted_topics = lda.transform(bag)

In [ ]:
reviews_df['TopicCategory'] = predicted_topics.argsort()[:,-1]
reviews_df[['asin', 'reviewText', 'summary', 'TopicCategory']]

,asin,reviewText,summary,TopicCategory
0,B0000V9K32,the colour i received is not blue as shown but...,not the colour,14
1,B0000V9K32,Very cute and is really practical. Fits better...,Cute and practical!,1
2,B0000V9K3W,The watch was exactly what i ordered and I got...,"Great watch, just too big",13
3,B0000V9K46,"This product came promptly and as described, p...",Just as described!,4
4,B0000V9KNM,Why isn't Nike making these anymore? I love t...,Great watch,1
...,...,...,...,...
21565,B00KRENKNS,love it,Five Stars,11
21566,B00KXBJQ1A,They fit perfect and the are really light,Five Stars,11
21567,B00L01ML5K,The product was shipped in record speed! I lov...,I love these Nikes,13
21568,B00L2ADWCA,I like this item but they didn't have my size ...,airmax,5


## Look at the topics, and matching reviews from the dataset

In [ ]:
for i in range(number_of_topics):
  print(f"Topic {i}: ", counter.get_feature_names_out()[lda.components_[i].argsort()[-15:][::-1]])
  with pd.option_context("display.max_colwidth", None):
    display(reviews_df[reviews_df['TopicCategory'] == i].sample(5)[['asin', 'reviewText', 'summary', 'TopicCategory']])

Topic 0:  ['cleat' 'white' 'like' 'play' 'black' 'nike' 'soccer' 'game' 'son' 'good'
 'basketball' 'shoe' 'day' 'season' 'practice']


,asin,reviewText,summary,TopicCategory
6470,B0042B4C7U,My son says they are a comfortable shoe for basketball. Plus our whole teams has these shoes and their are 25 kids on our JV team.,good shoes for basketball,0
2641,B001GUNBA4,The pic shows an orange silver and white running shoe. Then a yellow pair of shorts and the description now that I've ordered the shoes has changed to a maroon white and silver turf shoe? What's going on w this?,Confused,0
6546,B0045G7Q3O,"My son had his heart set on these sneakers, looked everywhere then found them here and the price was great too! Have had new for several months and I am very happy witht he quality, unlike some other brands, these are holding up beautifully! NIKE fan!",JUSt WHAT WE WANTED!,0
3338,B001YYKGK0,"Personally, I love how these sandals have the Nike wording on top of them. Unlike many Nike sandals I see around, this one doesn't just have the swoosh.",Nice lil sandals!,0
20759,B00FWXYOB6,"My son loves these cleats! They fit well, are comfortable and look great! He's just started conditioning but so far they've held up terrific. The additional cleats for extra traction are a nice feature.",Great shoes!,0


Topic 1:  ['watch' 'time' 'band' 'wrist' 'like' 'work' 'easy' 'look' 'use' 'nike'
 'battery' 'feature' 'face' 'great' 'buy']


,asin,reviewText,summary,TopicCategory
636,B000E8F27Y,I purchased this watch for my wif'e birthday. I was nervous about it because it was a Nike Watch ... not a Nike Shoe.My wife loves this watch. Her hands are in and out of water everyday on her job and because the watch is waterproof she doesn't have to worry about it. She loves the comfort of the band ... it doesn't irritate her wrist at all. Especially she loves the angle that the watch head itself sits on the band ... she doesn't even have to turn her wrist to see the time.I couldn't have made a better choice for her lifestyle. Thanks Nike/Amazon.,Nike Ladie's Watch,1
1169,B000J2D68W,"I was so exited when I recieved the watch 2 weeks ago but, and then I opened the box, the watch looked really nice as I expected but it didn't work so I have to return it the next day. I am still waiting for my refund.",The worst watch I ever bought,1
746,B000E8H0BA,"I never return items, but this is unusable. I can hardly read the display. I attempted to adjust the contrast setting, but it had no noticeable effect.I had a triax and love it. just dont get the black on black.",Too dark to read,1
759,B000E8H0BU,"Had this watch for almost 2 years. Was excellent for running and easy to use. Trained for a marathon and multiple 5ks with it. Now it says good to 10M but that is clearly not true. I wasn't 10ft underwater and the watch gave out. But If you don't plan to be diving in any pools or lakes, it is an excellent watch. I highly recommend it for beginning to experienced runners due to its simplicity.",don't take underwater,1
483,B000A8BU8S,"My son purchased this watch for himself. He loves it. It wears well, it's pretty rough & tough. He learned to set it and use it with a little help from Mom. Great value!",Great Watch!,1


Topic 2:  ['de' 'muy' 'el' 'la' 'que' 'en' 'para' 'los' 'es' 'lo' 'excelente'
 'producto' 'son' 'con' 'se']


...

,asin,reviewText,summary,TopicCategory
11061,B005BQXZXC,Poorly made. Visable glue. Probably counterfit Jordans,Poor mans Jordan's,17
14512,B007PKCTPC,Thank you very much hooded sweater is excellent my son was fascinated. with this purchaseI'm very comfortable good for this purchase and I congratulate,thank you very much,17
20887,B00GR4V0IY,"I wear size 7 or 8 depending, on the manufacturer. I purchased the size 14 (was hoping they had a larger pair). Either way, I am able to fit both my feet into one sneaker and my wife & son can squish their feet into the other. We hop around town while all others look on with jealousy all over their faces!",Best investment ever!,17
2988,B001OVSVS2,Did not like the sneakers. Were told they had a little yellowing on the trim due to be and out dated discontinued pair of sneakers. But when I got them home they were very noticeable yellowing. More then I expected. Would be very noticeable wearing them. And for 99$ to much yellowing for the money. Sent them back for a refund.,Yellow sneakers,17
18627,B00BMA1OLQ,perfectos,Five Stars,17


Topic 18:  ['shoe' 'comfortable' 'great' 'light' 'recommend' 'good' 'run' 'use'
 'look' 'weight' 'highly' 'play' 'wear' 'fit' 'gym']


,asin,reviewText,summary,TopicCategory
13352,B0070061OW,"this type of shoes is perfect for walking, working out in the gym (except running on the treadmill), and dancing. i have wide feet and usually some athletic shoes that i buy are one size bigger because of it. this one is different; it had just my size. the fabric won't suffocate your feet. it is recommended for indoor-type workout shoes. all in all, the shoes are fantastic.",comfort and function,18
15905,B0098G7Q1S,To small and not worth the price tag. Just get a good pair of Asic's that you can use for both the outdoors and the gym and you should be set. All these light shoes are not durable and after a year or so they're not so comfortable.,To small,18
17584,B00AMU1YB2,Great running shoe. Comfortable and lightweight. I'm very happy with my purchase and I will probably buy another pair :),Nike lunarglide love!,18
2844,B001LRNHPQ,It fits perfect. Very comfortable shoes. I recommend its use for trail running and also in any synthetic field sport.,Fit Perfect,18
15929,B0098G7Q1S,I use this shoe for my daily workout. It's light and feels good. Great product!!,It's light and feels good. Great product,18


Topic 19:  ['sock' 'fit' 'foot' 'like' 'wear' 'short' 'dry' 'great' 'good' 'look'
 'thick' 'wash' 'thin' 'stay' 'comfortable']


,asin,reviewText,summary,TopicCategory
2557,B001E4X1T8,These are probably the most comfortable boots I have ever purchased. It really is worth the money. I'm always on my feet and I love how its waterproof. Looks great with some boot cut jeans. Def worth the money. If it were up to me I would buy them again and again. Oh yeah another thing is that they are lightweight.,Very comfortable,19
5214,B0036DRWKO,This is one of my favorite bras! It fits well and holds its shape. I will be purchasing more in the future,Comfortable,19
6054,B003QL2SQO,"I like the sizing, fabric, breathability, and left / right refrenes of these socks. Black socks don't get the brown sweat and dirt stains that white socks do. These are really comfortable.",Great Socks!,19
4564,B002N2DP32,Awesome shorts. Great fit and very comfortable. It is very breathable which I like. A bit on the expensive side tho,Comfortable shorts,19
18938,B00C3VGNNC,I have a lot of NIKE Shoes......these fit especially tight because they are different in the fact that she have elastic not unlike a sock. The laces aren't really needed. THEY ARE LIGHTER than AIR. I love them.,SUPER LIGHTWEIGHT.... BUT VERY SNUG,19


Topic 2 is a collection of Nike reviews in different language (in this case mostly Spanish)  

Topics 1, 3, and 5, 10, 11, 18 are related to Nike Shoes generally.
Topic 9 is related to Nike Bags.
Topic 12 is related to Nike Sunglasses.
Topic 16 involves Nike Shirts and Socks.


Topic 4 is mostly positive responses to the product. These can be used for positive social media campaigns.  

Topic 7 is a mixed bag of reviews but has valid issues customers faced with the product. This can be used for **Product Insights** to improve future iterations of the product.

Topic 13 has a collection of **Size Issues** pretaining to both shoes and watches.

